In [6]:
from flowermd.base import Pack,  Simulation
from flowermd.library import PPS, OPLS_AA_PPS
from flowermd.utils import get_target_box_mass_density
import unyt as u
import hoomd

import numpy as np
import gsd.hoomd

import warnings
warnings.filterwarnings('ignore')

In [3]:
N_beads = 300

In [4]:
pps = PPS(num_mols=N_beads, lengths=1)
system = Pack(molecules=pps, density=0.3, fix_orientation=True)
system.apply_forcefield(r_cut=2.5, force_field=OPLS_AA_PPS(), auto_scale=True, remove_charges=True, scale_charges=True, remove_hydrogens=True)
frame = system.hoomd_snapshot

No charged group detected, skipping electrostatics.


In [16]:
target_density = 0.3 * u.Unit("g")/u.Unit("cm")**3
mass = system.mass.to("g")
target_box = get_target_box_mass_density(density=target_density, mass=mass)
target_box = (target_box.to('nm') / system.reference_length).value
target_box

array([15.76909943, 15.76909943, 15.76909943])

In [80]:
import pickle
with open("pps_ff.pkl", "rb") as f:
    pps_ff = pickle.load(f)

## Calculate relative positions and COM

In [73]:
init_frame = system.hoomd_snapshot
constituent_indx = [np.arange(7) + (i*7) for i in range(N_beads) ]


com_mass = []
com_positions = []
const_mass = []
const_pos = []
rel_pos = []

for indx in constituent_indx:
    m = np.asarray(init_frame.particles.mass)[indx]
    pos = np.asarray(init_frame.particles.position)[indx]
    
    const_mass.append(m)
    const_pos.append(pos)
    
    total_mass = np.sum(m)
    com_mass.append(total_mass)

    com_pos = np.sum(pos * m[:, np.newaxis], axis=0) / total_mass
    com_positions.append(com_pos)
    rel_pos.append(pos - com_pos)
    

In [74]:
rel_pos

[array([[ 0.        , -0.02527237,  0.6505661 ],
        [ 0.33646393, -0.01771927,  0.45647812],
        [ 0.3367157 , -0.00262451,  0.06819916],
        [ 0.        ,  0.00485229, -0.12650299],
        [ 0.        ,  0.02420044, -0.6233063 ],
        [-0.3367195 , -0.00262451,  0.06819916],
        [-0.33646774, -0.01771927,  0.45647812]], dtype=float32),
 array([[-1.9073486e-06, -2.5266647e-02,  6.5056419e-01],
        [ 3.3646393e-01, -1.7717361e-02,  4.5647430e-01],
        [ 3.3671570e-01, -2.6226044e-03,  6.8197250e-02],
        [-1.9073486e-06,  4.8542023e-03, -1.2650681e-01],
        [-1.9073486e-06,  2.4202347e-02, -6.2330818e-01],
        [-3.3671951e-01, -2.6206970e-03,  6.8197250e-02],
        [-3.3646774e-01, -1.7717361e-02,  4.5647430e-01]], dtype=float32),
 array([[ 0.        , -0.02526855,  0.6505642 ],
        [ 0.33646393, -0.01771927,  0.4564724 ],
        [ 0.3367157 , -0.00262451,  0.06819534],
        [ 0.        ,  0.00485229, -0.1265068 ],
        [ 0.        ,

In [75]:
rel_const_pos = rel_pos[0]
np.save('const_rel_pos.npy', rel_const_pos)

In [76]:
const_particle_types = list(np.asarray(frame.particles.types)[list(frame.particles.typeid)][constituent_indx[0]])
const_particle_types

['ca', 'ca', 'ca', 'ca', 'sh', 'ca', 'ca']

## Calculate Moment of Inertia

In [77]:
I = np.zeros(shape=(3, 3))

for r, mass in zip(rel_pos[1], const_mass[1]):
    I += mass * (np.dot(r, r) * np.identity(3) - np.outer(r, r))

In [78]:
[I[0, 0], I[1, 1], I[2, 2]]

[0.7527316916239202, 0.9356256943100995, 0.18515955004552742]

In [79]:
# create the init rigid lattice

rigid_frame = gsd.hoomd.Frame()
rigid_frame.particles.types = ['rigid', 'ca', 'sh']

N_rigid = len(com_mass)
rigid_frame.particles.N = N_beads
rigid_frame.particles.position = com_positions
rigid_frame.particles.typeid = [0] * N_beads
rigid_frame.configuration.box = init_frame.configuration.box
rigid_frame.particles.mass = com_mass
rigid_frame.particles.moment_inertia = np.tile([I[0, 0], I[1, 1], I[2, 2]], (N_beads, 1))
rigid_frame.particles.orientation = [(1, 0, 0, 0)] * N_beads

rigid = hoomd.md.constrain.Rigid()
rigid.body['rigid'] = {
    "constituent_types":const_particle_types,
    "positions": rel_const_pos,
    "orientations": [(1.0, 0.0, 0.0, 0.0)]* len(rel_const_pos),
    }
simulation = hoomd.Simulation(device=hoomd.device.CPU(), seed=4)
simulation.create_state_from_snapshot(rigid_frame)

rigid.create_bodies(simulation.state)
integrator = hoomd.md.Integrator(dt=0.005, integrate_rotational_dof=True)
integrator.rigid = rigid
simulation.operations.integrator = integrator
simulation.run(0)

hoomd.write.GSD.write(state=simulation.state, mode='wb', filename='rigid_init_{}.gsd'.format(N_beads))

## Create Rigid body simulation

In [91]:
dict(pps_ff[1].params)

{'ca-ca': _HOOMDDict{'epsilon': 0.08235294117647057, 'sigma': 0.9861111111111112},
 'ca-sh': _HOOMDDict{'epsilon': 0.20291986247835692, 'sigma': 0.9930312739844155}}

In [92]:
dict(pps_ff[0].params)

{('ca',
  'ca'): _HOOMDDict{'epsilon': 0.16470588235294115, 'sigma': 0.9861111111111112},
 ('ca',
  'sh'): _HOOMDDict{'epsilon': 0.40583972495671383, 'sigma': 0.9930312739844155},
 ('sh', 'sh'): _HOOMDDict{'epsilon': 0.9999999999999999, 'sigma': 1.0}}

In [119]:
def create_rigid_simulation(kT, init_snapshot, pps_ff, rel_const_pos, dt, tau, special_LJ=True, write_freq=1000):
    const_particle_types = ['ca', 'ca', 'ca', 'ca', 'sh', 'ca', 'ca']
    rigid_simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=1)
    rigid_simulation.create_state_from_snapshot(init_snapshot)


    rigid = hoomd.md.constrain.Rigid()
    rigid.body['rigid'] = {
        "constituent_types":const_particle_types,
        "positions": rel_const_pos,
        "orientations": [(1.0, 0.0, 0.0, 0.0)]* len(rel_const_pos),
        }
    integrator = hoomd.md.Integrator(dt=dt, integrate_rotational_dof=True)
    rigid_simulation.operations.integrator = integrator
    integrator.rigid = rigid
    rigid_centers_and_free = hoomd.filter.Rigid(("center", "free"))
    nvt = hoomd.md.methods.ConstantVolume(
        filter=rigid_centers_and_free,
        thermostat=hoomd.md.methods.thermostats.MTTK(kT=kT, tau=tau))
    integrator.methods.append(nvt)
    
    cell = hoomd.md.nlist.Cell(buffer=0, exclusions=['body'])
    
    lj = hoomd.md.pair.LJ(nlist=cell)
    
    # use aa pps simulation to define lj and special lj forces between constituent particles
    for k, v in dict(pps_ff[0].params).items():
        lj.params[k] = v
        lj.r_cut[k] = 2.5
    
    lj.params[('rigid', ['rigid', 'ca', 'sh'])]= dict(epsilon=0, sigma=0)
    lj.r_cut[('rigid', ['rigid', 'ca', 'sh'])] = 0

    integrator.forces.append(lj)
    if special_LJ:
        special_lj = hoomd.md.special_pair.LJ()
        for k, v in dict(pps_ff[1].params).items():
            special_lj.params[k] = v
            special_lj.r_cut[k] = 2.5
        
        special_lj.params[('rigid', ['rigid', 'ca', 'sh'])]= dict(epsilon=0, sigma=0)
        special_lj.r_cut[('rigid', ['rigid', 'ca', 'sh'])] = 0

        integrator.forces.append(special_lj)
        
        
    rigid_simulation.state.thermalize_particle_momenta(filter=rigid_centers_and_free,
                                             kT=kT)
    
    rigid_simulation.run(0)

    
    log_quantities = [
                        "kinetic_temperature",
                        "potential_energy",
                        "kinetic_energy",
                        "volume",
                        "pressure",
                        "pressure_tensor",
                    ]
    logger = hoomd.logging.Logger(categories=["scalar", "string", "particle"])
    logger.add(rigid_simulation, quantities=["timestep", "tps"])
    thermo_props = hoomd.md.compute.ThermodynamicQuantities(filter=rigid_centers_and_free)
    rigid_simulation.operations.computes.append(thermo_props)
    logger.add(thermo_props, quantities=log_quantities)
    
    # for f in integrator.forces:
    #     logger.add(f, quantities=["energy", "forces", "energies"])

    logger.add(rigid_simulation.operations.integrator.rigid, quantities=["torques", "forces", "energies"])
    
    gsd_writer = hoomd.write.GSD(
        filename="trajectory.gsd",
        trigger=hoomd.trigger.Periodic(int(write_freq)),
        mode="wb",
        logger=logger,
        filter=hoomd.filter.All(),
        dynamic=["momentum", "property"]
        )
    
    rigid_simulation.operations.writers.append(gsd_writer)

    table_logger = hoomd.logging.Logger(categories=["scalar", "string"])
    table_logger.add(rigid_simulation, quantities=["timestep", "tps"])
    table_logger.add(thermo_props, quantities=log_quantities)
    table_file = hoomd.write.Table(
            output=open("log.txt", mode="w", newline="\n"),
            trigger=hoomd.trigger.Periodic(int(write_freq)),
            logger=table_logger,
            max_header_len=None,
        )
    rigid_simulation.operations.writers.append(table_file)
    return rigid_simulation

    
def shrink_volume(simulation, kT, target_box, n_steps):
    # box resizer
    final_box = hoomd.Box(
            Lx=target_box,
            Ly=target_box,
            Lz=target_box,
        )
    resize_trigger = hoomd.trigger.Periodic(100)
    box_ramp = hoomd.variant.Ramp(
        A=0, B=1, t_start=simulation.timestep, t_ramp=int(n_steps)
    )
    initial_box = simulation.state.box

    box_resizer = hoomd.update.BoxResize(
        box1=initial_box,
        box2=final_box,
        variant=box_ramp,
        trigger=resize_trigger,
    )
    simulation.operations.updaters.append(box_resizer)
    simulation.run(n_steps + 1, write_at_start=True)
    simulation.operations.updaters.remove(box_resizer)

    
    

In [120]:
init_snapshot = gsd.hoomd.open("rigid_init_300.gsd")[-1]
rel_const_pos = np.load("const_rel_pos.npy")

rigid_sim = create_rigid_simulation(kT=1.0, init_snapshot=init_snapshot, pps_ff=pps_ff, rel_const_pos=rel_const_pos, 
                                    dt=0.0001, tau=0.1, special_LJ=False, write_freq=1e4)

In [121]:
shrink_volume(rigid_sim, kT=1.0, target_box=target_box[0], n_steps=1e5)

In [128]:
target_box

array([15.76909943, 15.76909943, 15.76909943])

In [124]:
rigid_sim.run(4e5)

In [117]:
init_snapshot.particles.types

['rigid', 'ca', 'sh']

In [125]:
for writer in rigid_sim.operations.writers:
    if hasattr(writer, 'flush'):
        writer.flush()

In [126]:
log = np.genfromtxt("log.txt", names=True)

potential_energy = log['mdcomputeThermodynamicQuantitiespotential_energy']

In [127]:
potential_energy

array([15125.   ,  -209.939,  -180.113,  -205.244,  -212.778,  -259.322,
        -285.263,  -332.827,  -403.512,  -698.317, -1593.11 , -1780.46 ,
       -2059.26 , -2243.08 , -2516.88 , -2701.78 , -2951.12 , -3076.27 ,
       -3223.91 , -3270.91 , -3274.2  , -3340.11 , -3372.42 , -3446.74 ,
       -3509.54 , -3523.26 , -3494.07 , -3574.04 , -3635.31 , -3672.62 ,
       -3662.36 , -3724.63 , -3688.4  , -3783.57 , -3712.28 , -3695.91 ,
       -3682.81 , -3672.55 , -3728.66 , -3765.25 , -3812.64 , -3733.02 ,
       -3687.74 , -3747.55 , -3760.51 , -3684.82 , -3729.5  , -3762.31 ,
       -3686.67 , -3745.26 , -3721.51 , -3758.09 , -3774.27 , -3767.78 ,
       -3708.87 , -3793.5  , -3830.19 , -3760.72 , -3863.01 , -3874.42 ,
       -3774.15 ])